In [1]:
pip install transformers datasets accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
!accelerate config

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--------------------------------------------------------------------------------In which compute environment are you running?
Please select a choice using the arrow or number keys, and selecting with enter
 ➔  This machine
    AWS (Amazon SageMaker)
Traceback (most recent call last):
  File "/home/vitoupro/code/image_captioning/image_env/bin/accelerate", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/vitoupro/code/image_captioning/image_env/lib/python3.12/site-packages/accelerate/commands/accelerate_cli.py", line 50, in main
    args.func(args)
  File "/home/vitoupro/code/image_captioning/image_env/lib/python3.12/site-packages/accelerate/commands/config/config.py", line 67, in config_command
    config = get_user_input()
             ^^^^^^^^^^^^^^^^
  File "/home/vitoupro/code/image_captioning/image_env/lib/python3.12/site-packages/accelerate/commands/config/config.py", line 32, in get_user_input
    compute_environment = _ask_options(
                     

In [8]:
import os
import pandas as pd
from datasets import Dataset

# === Set your paths ===
images_dir = '/home/vitoupro/code/image_captioning/data/raw/img'
captions_file = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel.txt'  # the file you have

# === Read captions file ===
image_paths = []
captions = []

with open(captions_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            img_name, caption = parts
            img_path = os.path.join(images_dir, img_name)
            if os.path.exists(img_path):
                image_paths.append(img_path)
                captions.append(caption)
            else:
                print(f"[!] Warning: Image {img_name} not found!")

# === Create dataset ===
data = {'image_path': image_paths, 'caption': captions}
df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)

print("✅ Loaded dataset:", len(dataset), "samples")


✅ Loaded dataset: 425 samples


In [1]:
import torch
from transformers import Blip2ForConditionalGeneration, Blip2Processor, TrainingArguments, Trainer
from datasets import Dataset
import os
from PIL import Image
from torchvision import transforms

model_name = "Salesforce/blip2-opt-2.7b"
processor = Blip2Processor.from_pretrained(model_name)
model = Blip2ForConditionalGeneration.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)


/home/vitoupro/code/image_captioning/image_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.66s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
# Paths
images_dir = '/home/vitoupro/code/image_captioning/data/raw/img'
captions_file = '/home/vitoupro/code/image_captioning/data/processed/word_segmented_imglabel.txt'

# Read
image_paths = []
captions = []

with open(captions_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        if len(parts) == 2:
            img_name, caption = parts
            img_path = os.path.join(images_dir, img_name)
            if os.path.exists(img_path):
                image_paths.append(img_path)
                captions.append(caption)
            else:
                print(f"[!] Warning: {img_name} not found!")

dataset = Dataset.from_dict({"image_path": image_paths, "caption": captions})


In [3]:
from torchvision import transforms

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def preprocess_data(example):
    image = Image.open(example['image_path']).convert('RGB')
    pixel_values = image_transform(image)

    inputs = processor(images=pixel_values, text=example['caption'], return_tensors="pt", padding="max_length", truncation=True)
    inputs = {k: v.squeeze(0) for k, v in inputs.items()}
    inputs['labels'] = inputs['input_ids']
    return inputs

processed_dataset = dataset.map(preprocess_data, remove_columns=dataset.column_names)


Map:   0%|          | 0/425 [00:00<?, ? examples/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
Map: 100%|██████████| 425/425 [00:06<00:00, 63.39 examples/s] 


In [6]:
pip install --upgrade transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 7.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.0
    Uninstalling transformers-4.51.0:
      Successfully uninstalled transformers-4.51.0

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
training_args = TrainingArguments(
    output_dir="./blip2-khmer-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    save_steps=100,
    save_total_limit=2,
    logging_steps=10,
    learning_rate=1e-5,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,  # You need to prepare this
)


You shouldn't move a model that is dispatched using accelerate hooks.


RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.